In [1]:
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [2]:
# load foursquare credentials
foursquare_cred = pd.read_csv('foursquare_cred') # saved locally and ignored in .gitignore

CLIENT_ID = foursquare_cred.CLIENT_ID[0] # your Foursquare ID
CLIENT_SECRET = foursquare_cred.CLIENT_SECRET[0] # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

LocationIQ api docs - https://locationiq.com/docs#forward-geocoding   

Region 1: US  
GET https://us1.locationiq.com/v1/search.php?key=YOUR_PRIVATE_TOKEN&q=SEARCH_STRING&format=json

Region 2: Europe    
GET https://eu1.locationiq.com/v1/search.php?key=YOUR_PRIVATE_TOKEN&q=SEARCH_STRING&format=json


In [6]:
# LocationIQ apikey
apikey = pd.read_table('locationiq_api_key',header=None).iloc[0,0]

In [10]:
# Search / Forward Geocoding url
query = 'Paris'
region = 'eu' # us or eu
search_url = "https://{}1.locationiq.com/v1/search.php".format(region)
data = {'key': apikey,'q': query,'format': 'json'}
LIQresponse = requests.get(search_url, params=data)

In [11]:
LIQresponse.json()

[{'boundingbox': ['48.8155755', '48.902156', '2.224122', '2.4697602'],
  'class': 'place',
  'display_name': 'Paris, Ile-de-France, Metropolitan France, France',
  'icon': 'https://locationiq.org/static/images/mapicons/poi_place_city.p.20.png',
  'importance': 0.97893459932191,
  'lat': '48.8566101',
  'licence': '© LocationIQ.com CC BY 4.0, Data © OpenStreetMap contributors, ODbL 1.0',
  'lon': '2.3514992',
  'osm_id': '7444',
  'osm_type': 'relation',
  'place_id': '200333750',
  'type': 'city'},
 {'boundingbox': ['48.8155755', '48.902156', '2.224122', '2.4697602'],
  'class': 'boundary',
  'display_name': 'Paris, Ile-de-France, Metropolitan France, France',
  'icon': 'https://locationiq.org/static/images/mapicons/poi_boundary_administrative.p.20.png',
  'importance': 0.97893459932191,
  'lat': '48.85881005',
  'licence': '© LocationIQ.com CC BY 4.0, Data © OpenStreetMap contributors, ODbL 1.0',
  'lon': '2.32003101155031',
  'osm_id': '71525',
  'osm_type': 'relation',
  'place_id':

In [12]:
# Search / Forward Geocoding url
city = 'Paris'
country = 'France'
region = 'eu' # us or eu
search_url = "https://{}1.locationiq.com/v1/search.php".format(region)
data = {'key': apikey,'city': city,'country':country,'format': 'json'}
LIQresponse = requests.get(search_url, params=data)

In [19]:
LIQresponse.json()[0]['display_name']

'Paris, Ile-de-France, Metropolitan France, France'

In [38]:
## http://code.activestate.com/recipes/52308-the-simple-but-handy-collector-of-a-bunch-of-named/?in=user-97991
class Bunch:
    def __init__(self, **kwds):
        self.__dict__.update(kwds)
def CityAtt(response):
    # response from LocationIQ in json
    tmp = Bunch(name= LIQresponse.json()[0]['display_name'], 
          lat= float(LIQresponse.json()[0]['lat']), 
          lon= float(LIQresponse.json()[0]['lon']))
    return tmp

In [39]:
Paris = CityAtt(LIQresponse)

In [40]:
Paris.name,Paris.lat,Paris.lon

('Paris, Ile-de-France, Metropolitan France, France', 48.8566101, 2.3514992)

In [41]:
# map
lat = Paris.lat
lon = Paris.lon
Fmap = folium.Map(location=[float(lat), float(lon)], zoom_start=10 , control_scale = True)
Fmap

In [43]:
lat = Paris.lat
lng = Paris.lon
radius = 8000 # meters, max = 100,000 ## https://developer.foursquare.com/docs/api/venues/search
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET,VERSION,lat,lng,radius,LIMIT)
# make the GET request
FSresults = requests.get(url).json()#["response"]['groups'][0]['items']

In [47]:
FSresults['response']['groups'][0]['items']

[{'reasons': {'count': 0,
   'items': [{'reasonName': 'globalInteractionReason',
     'summary': 'This spot is popular',
     'type': 'general'}]},
  'referralId': 'e-0-4bf41231e5eba59334341f90-0',
  'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/plaza_',
      'suffix': '.png'},
     'id': '4bf58dd8d48988d164941735',
     'name': 'Plaza',
     'pluralName': 'Plazas',
     'primary': True,
     'shortName': 'Plaza'}],
   'id': '4bf41231e5eba59334341f90',
   'location': {'address': "Place de l'Hôtel de Ville",
    'cc': 'FR',
    'city': 'Paris',
    'country': 'France',
    'distance': 60,
    'formattedAddress': ["Place de l'Hôtel de Ville", '75004 Paris', 'France'],
    'lat': 48.85713761176484,
    'lng': 2.3516931321499746,
    'postalCode': '75004',
    'state': 'Île-de-France'},
   'name': "Place de l'Hôtel de Ville – Esplanade de la Libération",
   'photos': {'count': 0, 'groups': []}}},
 {'reasons': {'count': 0,
   'items': [